In [1]:
import planetary_computer
import pystac_client
import xarray as xr
import fsspec
import numpy as np
import pandas as pd

In [2]:
## 可能需要安装以下组件以免报错：
# pip install aiohttp
# pip install h5netcdf

In [3]:
## Loading initial data
## SA有257个，WS有300个
data = pd.read_csv('Training_data_SA.csv')
data

,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Rice Yield (kg/ha)
0,Chau_Phu,10.510542,105.248554,SA,T,15-07-2022,3.40,5500
1,Chau_Phu,10.509150,105.265098,SA,T,15-07-2022,2.43,6000
2,Chau_Phu,10.467721,105.192464,SA,D,15-07-2022,1.95,6400
3,Chau_Phu,10.494453,105.241281,SA,T,15-07-2022,4.30,6000
4,Chau_Phu,10.535058,105.252744,SA,D,14-07-2022,3.30,6400
...,...,...,...,...,...,...,...,...
252,Thoai_Son,10.364419,105.164984,SA,T,12-07-2022,7.80,5600
253,Thoai_Son,10.358727,105.222371,SA,T,21-07-2022,2.00,5600
254,Thoai_Son,10.358094,105.189541,SA,T,21-07-2022,2.00,5600
255,Thoai_Son,10.368014,105.238516,SA,T,21-07-2022,6.20,5600


In [4]:
catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=planetary_computer.sign_inplace,
)

In [5]:
def get_bbox(lat: float, long: float, box_size_ha: float = 4):

    box_size_deg = box_size_ha * .001 * 0.98480775

    # cos 10° = 0.98480775
    # 1经度在赤道相当于111,000米， 1经度在纬度10°相当于111,000*0.98480775
    # 1公顷相当于100米×100米
    #
    # 对于赤道，图上1.11米相当于0.00001经度， 1公顷相当于0.001经度×0.001经度
    # 对于非赤道，1公顷相当于(0.001经度*cos纬度)×(0.001经度*cos纬度)

    lat_long = (lat, long) # Lat-Lon centroid location
    box_size_deg = box_size_deg # Surrounding box in degrees, yields approximately 5x5 pixel region

    min_lon = lat_long[1]-box_size_deg/2
    min_lat = lat_long[0]-box_size_deg/2
    max_lon = lat_long[1]+box_size_deg/2
    max_lat = lat_long[0]+box_size_deg/2

    bbox = [min_lon, min_lat, max_lon, max_lat]

    return bbox

In [6]:
def get_time_window(season: str = ['SA', 'WS', 'AW']):
    if season == 'SA':
        time_window = "2022-04-01/2022-08-31" #153天
        start_time = "2022-04-01"
        end_time = "2022-08-31"
    elif season == 'WS':
        time_window = "2021-11-01/2022-04-30" #181天
        start_time = "2021-11-01"
        end_time = "2022-04-30"
    elif season == 'AW':
        time_window = "2022-07-01/2022-12-31"
        start_time = "2022-07-01"
        end_time = "2022-12-31"
    else:
        print(f"{season} is not usable.")

    return time_window, start_time, end_time

In [7]:
# get items
def get_items(
        time_window: str,
        bbox: list,
        collections: list = ["nasa-nex-gddp-cmip6"],
        query: dict = {"cmip6:model": {"eq": "ACCESS-CM2"},"cmip6:scenario": {"eq": "ssp245"}}):

    search = catalog.search(
    collections = collections,
    datetime = time_window,
    bbox = bbox,
    query = query
    )

    items = search.get_all_items()
    print(f"Found {len(items)} items.")

    return items

In [8]:
%%time
#Load_data —— containing 9 variables:

# hurs (Description: Near-Surface Relative Humidity, Unit:%)
# huss (Description: Near-Surface Specific Humidity, Unit:1)
# pr (Description: Precipitation, Unit:kg/(m2*s))
# rlds (Description: Surface Downwelling Longwave Radiation, Unit:W/m2)
# rsds (Description: Surface Downwelling Shortwave Radiation, Unit:W/m2),
# sfcwind (Description: Daily-Mean Near-Surface Wind Speed, Unit:m*s),
# tas (Description: Daily Near-Surface Air Temperature, Unit:K),
# tasmax (Description: Daily Maximum Near-Surface Air Temperature, Unit:K),
# tasmin (Description: Daily Minimum Near-Surface Air Temperature, Unit:K)

variables = ["hurs","huss","pr","rlds","rsds","sfcWind","tas","tasmax","tasmin"]

#Produce a three-dimentional array to load the time-series-mean results of 9 variables for SA and WS respectively
timeseries_SA = np.zeros((257,153,9), dtype=float) #数组的三个维度分别代表 样本序号、日期、气候变量
timeseries_WS = np.zeros((300,181,9), dtype=float)

for i in range(len(data)):

    bbox = get_bbox(data['Latitude'][i], data['Longitude'][i], data['Field size (ha)'][i])
    time_window, start_time, end_time = get_time_window(data['Season(SA = Summer Autumn, WS = Winter Spring)'][i])
    items = get_items(time_window,bbox,)
    tgt_lat = xr.DataArray(np.arange(bbox[1],bbox[3],0.25)) #经纬度范围，一个像素点单位为 0.25 经纬度
    tgt_log = xr.DataArray(np.arange(bbox[0],bbox[2],0.25))

    for var in variables:
        exec("%s_ds = xr.open_mfdataset([fsspec.open(item.assets[var].href).open() for item in items],)" %var)
        exec("%s_da = %s_ds[var]" %(var,var))
        exec("%s_da = %s_da.loc[start_time:end_time]" %(var,var))
        exec("%s_da = %s_da.sel(**{'lat': slice(0.25*int((bbox[1]+59.875)/0.25)-59.875,0.25*int((bbox[3]+59.875)/0.25)-59.875), 'lon': slice(0.25*int((bbox[0]-0.125)/0.25)+0.125,0.25*int((bbox[2]-0.125)/0.25)+0.125)})" %(var,var)) ##四舍五入处理以满足要求
        exec("%s_mean = %s_da.groupby('time').mean(dim=['lat','lon']).values" %(var,var))
        exec("%s_mean = np.reshape(%s_mean,(-1,1))" %(var,var))

    ## exec("sample_%s = np.concatenate((hurs_da.values,huss_da.values,pr_da.values,rlds_da.values,rsds_da.values,sfcWind_da.values,tas_da.values,tasmax_da.values,tasmin_da.values),axis=1)"%i)
    
    temp = np.concatenate((hurs_mean,huss_mean,pr_mean,rlds_mean,rsds_mean,sfcWind_mean,tas_mean,tasmax_mean,tasmin_mean),axis=1)

    if data['Season(SA = Summer Autumn, WS = Winter Spring)'][i] == 'SA':
        timeseries_SA[i,:,:] = temp

    else:
        timeseries_WS[i,:,:] = temp

    print(f"the {i+1}th/{len(data)} sample is accomplished")

Found 1 items.
the 1th/257 sample is accomplished
Found 1 items.
the 2th/257 sample is accomplished
Found 1 items.
the 3th/257 sample is accomplished
Found 1 items.
the 4th/257 sample is accomplished
Found 1 items.
the 5th/257 sample is accomplished
Found 1 items.
the 6th/257 sample is accomplished
Found 1 items.
the 7th/257 sample is accomplished
Found 1 items.
the 8th/257 sample is accomplished
Found 1 items.
the 9th/257 sample is accomplished
Found 1 items.
the 10th/257 sample is accomplished
Found 1 items.
the 11th/257 sample is accomplished
Found 1 items.
the 12th/257 sample is accomplished
Found 1 items.
the 13th/257 sample is accomplished
Found 1 items.
the 14th/257 sample is accomplished
Found 1 items.
the 15th/257 sample is accomplished
Found 1 items.
the 16th/257 sample is accomplished
Found 1 items.
the 17th/257 sample is accomplished
Found 1 items.
the 18th/257 sample is accomplished
Found 1 items.
the 19th/257 sample is accomplished
Found 1 items.
the 20th/257 sample is ac

In [9]:
np.savez('results\weather_data', SA= timeseries_SA, WS= timeseries_WS)

In [ ]:
## Vietnam
# latitude = 10.5
# longitude = 105.2
# buffer = 0.1
# bbox = [longitude - buffer, latitude - buffer, longitude + buffer, latitude + buffer]
# year = "2022"
# months = {
#     "January": "01",
#     "February": "02",
#     "March": "03",
#     "April": "04",
#     "May": "05",
#     "June": "06",
#     "July": "07"
# }
# items = dict()
# months.items()

In [ ]:
## Fetch the collection of interest and print available items
# for name, number in months.items():
#     datetime = f"{year}-{number}"
#     search = catalog.search(
#         collections=["modis-11A1-061"],
#         bbox=bbox,
#         datetime=datetime,
#     )
#     items[name] = search.get_all_items()[0]
#
# print(items)

In [ ]:
# data_Day = odc.stac.load(
#     items.values(),
#     crs="EPSG:3857",
#     bands="LST_Day_1km",
#     resolution=500,
#     bbox=bbox,
# )
#
# raster_Day = items["March"].assets["LST_Day_1km"].extra_fields["raster:bands"]
# data_Day = data_Day["LST_Day_1km"] * raster_Day[0]["scale"]
# data_Day